In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.chdir('/content/drive/MyDrive/VinBrain/DMEC/Demo/CFS_v2')

In [5]:
!pip install -r requirements.txt

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 169.1 MB 45 kB/s 
     |████████████████████████████████| 13.3 MB 68 kB/s 
     |████████████████████████████████| 7.1 MB 4.7 MB/s 
     |████████████████████████████████| 28.2 MB 58 kB/s 
     |████████████████████████████████| 2.2 MB 41.0 MB/s 
     |████████████████████████████████| 189 kB 63.0 MB/s 
     |████████████████████████████████| 108.9 MB 31 kB/s 
     |████████████████████████████████| 361.4 MB 23 kB/s 
     |████████████████████████████████| 6.4 MB 53.5 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 7.9 MB 36.8 MB/s 
     |████████████████████████████████| 123 kB 70.9 MB/s 
     |████████████████████████████████| 12.5 MB 16 kB/s 
     |████████████████████████████████| 2.7 MB 46.7 MB/s 
     |████████████████████████████████| 50 kB 5.8 MB/s 
     |████████████████████████████████| 948 kB 36.2 MB/s 
     |█████

In [6]:
!pip install pytesseract

  Created wheel for pytesseract: filename=pytesseract-0.3.8-py2.py3-none-any.whl size=14071 sha256=a8ea1566307b345b5def2348af313b12f2d06cb8596e14b2769a46ead7c8d2c3
  Stored in directory: /root/.cache/pip/wheels/a4/89/b9/3f11250225d0f90e5454fcc30fd1b7208db226850715aa9ace
Successfully built pytesseract


In [7]:
!pip install fuzzywuzzy

In [8]:
!pip install datefinder

In [9]:
!pip install onnxruntime

     |████████████████████████████████| 4.5 MB 5.1 MB/s 


In [1]:
%%capture

# Install txtai, datasets and elasticsearch python client
!pip install git+https://github.com/neuml/txtai datasets elasticsearch

# Download and extract elasticsearch
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.10.1-linux-x86_64.tar.gz
!tar -xzf elasticsearch-7.10.1-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.10.1

In [2]:
import os
from subprocess import Popen, PIPE, STDOUT

# If issues are encountered with this section, ES can be manually started as follows:
# ./elasticsearch-7.10.1/bin/elasticsearch

# Start and wait for server
server = Popen(['elasticsearch-7.10.1/bin/elasticsearch'], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1))
!sleep 30

In [ ]:
!python CFS.py

In [ ]:
!python implement.py

2021-07-30 04:06:25.642612: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/usr/local/lib/python3.7/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  "Available providers: '{}'".format(name, ", ".join(available_provider_names)))

https://dmec.moh.gov.vn/documents/10182/12835224/upload_00731769_1573628012056.pdf?version=1.0&fileId=12847339
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)

Namespace(batch_size=100, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/root/.paddleocr/2.1/cls', cls_thresh=0.9, det=Fa

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import sys
from  PIL  import Image

img = cv.imread('/content/test.jpg', cv.IMREAD_UNCHANGED)
original = img.copy()

l = int(max(5, 6))
u = int(min(6, 6))

ed = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
edges = cv.GaussianBlur(img, (21, 51), 3)
edges = cv.cvtColor(edges, cv.COLOR_BGR2GRAY)
edges = cv.Canny(edges, l, u)

_, thresh = cv.threshold(edges, 0, 255, cv.THRESH_BINARY  + cv.THRESH_OTSU)
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
mask = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel, iterations=4)

data = mask.tolist()
sys.setrecursionlimit(10**8)
for i in  range(len(data)):
    for j in  range(len(data[i])):
        if data[i][j] !=  255:
            data[i][j] =  -1
        else:
            break
    for j in  range(len(data[i])-1, -1, -1):
        if data[i][j] !=  255:
            data[i][j] =  -1
        else:
            break
image = np.array(data)
image[image !=  -1] =  255
image[image ==  -1] =  0

mask = np.array(image, np.uint8)

print(mask.shape)
print(original.shape)

result = cv.bitwise_and(original, original, mask=mask)
result[mask ==  0] =  255
cv.imwrite('/content/bg.png', result)

img = Image.open('/content/bg.png')
img.convert("RGBA")
datas = img.getdata()

newData = []
for item in datas:
    if item[0] ==  255  and item[1] ==  255  and item[2] ==  255:
        newData.append((255, 255, 255, 0))
    else:
        newData.append(item)

img.putdata(newData)
img.save("/content/img.png", "PNG")

(743, 880)
(743, 880, 3)


AttributeError: ignored

In [ ]:
a = [[1,2],[2,3]]
a[:,1]

TypeError: ignored

In [ ]:
def push_result(es, result):
    buffer = []
    x=0
    for res in result:
        for text in res: 
            # Article record
            article = {"_id": x, "_index": "articles", "title": text}
            x+=1
            # Buffer article
            buffer.append(article)
    print(buffer)
    if buffer:
        helpers.bulk(es, buffer)


def search(query, limit):
    query = {
        "size": limit,
        "query": {
            "query_string": {"query": query}
        }
    }

    results = []
    for result in es.search(index="articles", body=query)["hits"]["hits"]:
        print(result)
        
        source = result["_source"]
        results.append((min(result["_score"], 18), source["title"]))

    return results

In [ ]:
from elasticsearch import Elasticsearch, helpers

es = Elasticsearch(hosts=["http://localhost:9200"], timeout=60, retry_on_timeout=True)


In [ ]:
es.indices.refresh("articles")
es.cat.count("articles", params={"format": "json"})

[{'count': '5', 'epoch': '1627377051', 'timestamp': '09:10:51'}]

In [ ]:
result = [[' CELL MARQUE CORP . Legal Manufacturer and Distributor page 9'], [' 760-4528 06419194001   Melanoma Associated Antigen (PNL2) Mouse Monoclonal Antibody page 13', ' 760-439005463572001 MUCG (MRQ-20) Mouse Monoclonal Antibody page 13', ' 760-2659   05267692001   Myeloperoxidase Rabbit Polyclonal Antibody page 13', ' 760-4391   05463599001 NGFR (Nerve Growth Factor Receptor) (MRQ-21) Mouse Monoclonal Antibody page 13']]
push_result(es, result)
query = 'CELL MARQUE CORP'
result = search(query,3)
print(result)

[{'_id': 0, '_index': 'articles', 'title': ' CELL MARQUE CORP . Legal Manufacturer and Distributor page 9'}, {'_id': 1, '_index': 'articles', 'title': ' 760-4528 06419194001   Melanoma Associated Antigen (PNL2) Mouse Monoclonal Antibody page 13'}, {'_id': 2, '_index': 'articles', 'title': ' 760-439005463572001 MUCG (MRQ-20) Mouse Monoclonal Antibody page 13'}, {'_id': 3, '_index': 'articles', 'title': ' 760-2659   05267692001   Myeloperoxidase Rabbit Polyclonal Antibody page 13'}, {'_id': 4, '_index': 'articles', 'title': ' 760-4391   05463599001 NGFR (Nerve Growth Factor Receptor) (MRQ-21) Mouse Monoclonal Antibody page 13'}]
{'_index': 'articles', '_type': '_doc', '_id': '0', '_score': 5.1123023, '_source': {'title': ' CELL MARQUE CORP . Legal Manufacturer and Distributor page 9'}}
[(5.1123023, ' CELL MARQUE CORP . Legal Manufacturer and Distributor page 9')]


In [ ]:
query = '760-4528'
result = search(query,3)
print(result)

{'_index': 'articles', '_type': '_doc', '_id': '1', '_score': 1.6139534, '_source': {'title': ' 760-4528 06419194001   Melanoma Associated Antigen (PNL2) Mouse Monoclonal Antibody page 13'}}
{'_index': 'articles', '_type': '_doc', '_id': '3', '_score': 0.3107994, '_source': {'title': ' 760-2659   05267692001   Myeloperoxidase Rabbit Polyclonal Antibody page 13'}}
{'_index': 'articles', '_type': '_doc', '_id': '2', '_score': 0.29879427, '_source': {'title': ' 760-439005463572001 MUCG (MRQ-20) Mouse Monoclonal Antibody page 13'}}
[(1.6139534, ' 760-4528 06419194001   Melanoma Associated Antigen (PNL2) Mouse Monoclonal Antibody page 13'), (0.3107994, ' 760-2659   05267692001   Myeloperoxidase Rabbit Polyclonal Antibody page 13'), (0.29879427, ' 760-439005463572001 MUCG (MRQ-20) Mouse Monoclonal Antibody page 13')]


In [ ]:
#CFS
import fitz
from utils import *
import json
import numpy as np
import time 
# from txtai.pipeline import Similarity

from elasticsearch import Elasticsearch, helpers

es = Elasticsearch(hosts=["http://localhost:9200"], timeout=60, retry_on_timeout=True)

class CFS(object):

    def __init__(self):
        self.pdf_path = None
        self.is_local = True
        self.info = {}
        self.reader = {
            'Tool' : self.tool_reader,
            'AI' : self.deeplearning_reader
        }
        self.pdf_info = {}

    def tool_reader(self, pdf_path):
        pdf_info = {}
        doc = fitz.open(pdf_path)
        p = 0
        for page in doc:
            pdf_info[f'page_{p + 1}'] = []
            blocks = json.loads(page.getText('json'))['blocks']
            blocks = sort_block(blocks)
            for block in blocks:
                text = get_text(block)
                pdf_info[f'page_{p + 1}'].append(text)
            p += 1
        return pdf_info

    def deeplearning_reader(self, pdf_path, check_annot=False):
        import easyocr
        reader = easyocr.Reader(['vi','en'], verbose=False, gpu = True)
        # doc = fitz.open(pdf_path)
        # p = 0
        pdf_info = {}
        # for page in doc:
        #     img = convert_img(page)
        #     img = np.array(img)
        #     result = reader.readtext(img, paragraph = True)
        #     pdf_info[f"page_{p + 1}"] = result
        #     p += 1
        p = 0
        t1 = time.time()
        images, annots = pdfimage_process(pdf_path, check_annot=check_annot)
        print(time.time()-t1)
        # print(annots)
        # exit()
        results = []
        horizontal_results = []
        if check_annot:
            for p, annot in enumerate(annots):
                if annot is not None:
                    print('=='*30)
                    # print(annot)
                    result, horizontal_result = reader.readtext(annot, paragraph=False, cluster=True, page=p+1)
                    if result:
                        results.append(result)
                    if horizontal_result:
                        horizontal_results.append(horizontal_result)
                    # pdf_info[f"page_{p + 1}"] = result
                p += 1
        # print(horizontal_results)
        # for img in images:
        #     img = np.array(img)
        #     result = reader.readtext(img, paragraph = True)
        #     pdf_info[f"page_{p + 1}"] = result
        #     p += 1
        return results, horizontal_results

    def get_ptn_info(self, pdf_info):
        result = {
            "registrant" : {
                "name" : "",
                "address" : ""
            },
            "factory" : {
                "name" : "",
                "address" : "",
            },
            "equipment" : ""
        }

        for page in pdf_info.keys():
            flag = False
            for text in pdf_info[page]:
                print(text)
                print("----------------------")
                if "1. Tên cơ sơ công bố" in text:
                    result['registrant']['name'] = text.split(':')[-1].strip()
                elif "2. Địa chỉ" in text:
                    result['registrant']['address'] = text.split('(')[0].split(':')[-1]
                if '(Sản xuất tại:' in text:
                    result['factory']['name'] = text.split('(')[-1].strip().split(";")[0].split(":")[-1].strip()
                    result['factory']['address'] = text.split(";")[-1].replace(')', '')
                elif "5. Tên trang thiết bị" in text:
                    flag = True
                    continue
                if flag:
                    result['equipment'] = text
                    flag = False
        return result

     
    def get_info(self, info):
        if isinstance(info, str):
            info = json.load(open(info, 'w'))
        
        key_info = {
            "equipment" : info['equipment'],
            "equipmentOwner" : info['equipmentOwner']
        }
        if len(info['cfsForeign']) > 0 and len(info['cfsLocal']) == 0:
            self.is_local = False

        
        if self.is_local:
            urls = info['cfsLocal']['files']
            for url in urls:
                pdf_path = save_pdf(url['url'])
                try:
                    pdf_info = self.reader['Tool'](pdf_path)
                except:
                    pdf_info = self.reader['AI'](pdf_path)
                self.pdf_info[f"file_{url[id]}"] = pdf_info


        else:
            urls = info['cfsForeign']['files']
            for url in urls:
                pdf_path = save_pdf(url['url'])
                pdf_info = self.reader['AI'](pdf_path)
                self.pdf_path[f'file_{url[id]}'] = pdf_info




if __name__ == '__main__':

    cfs = CFS()
    pdf_path = 'test1.pdf'
    # pdf_info = cfs.tool_reader(pdf_path)
    # result = cfs.get_ptn_info(pdf_info)
    # print(result)

    es = Elasticsearch(hosts=["http://localhost:9200"], timeout=60, retry_on_timeout=True)

    vertical_results, horizontal_results = cfs.deeplearning_reader(pdf_path)
    print(vertical_results)
    push_result(es, vertical_results)
    query = '760-4528'
    result = search(es,query,1)
    print(result)



    # print(pdf_info)



In [23]:
import re

a = "nnahda dagaf dả rá ' sfsdr dẳef"
nstr = re.sub(r"'",r'',str(a))
print(nstr)


nnahda dagaf dả rá  sfsdr dẳef


In [45]:
from dateutil import parser as pr
time_end = 'Soclate Anonyme a Dlrerioire el Conseil de Surveillance Signature : Date :21/11/2019 au capital de 20 394 496 Euros page 1'
time_end = pr.parse(time_end.strip())
print(time_end)

2019-11-21 00:00:00


In [47]:
import re
from datetime import datetime
text = 'Soclate Anonyme a Dlrerioire el Conseil de Surveillance Signature : Date :21/11/2019 au capital de 20/7/2021 Euros page 1'
match = re.search(r'\d{4}-\d{2}-\d{2}', text)
print(match)
date = datetime.strptime(match.group(), '%Y-%m-%d').date()

None


AttributeError: ignored

In [71]:
import datefinder
import datetime

input_string = 'Soclate Anonyme a Dlrerioire el Conseil de Surveillance Signature : Date :21/11/2019 au capital de 20 394 496 Euros 20/7/2021 page 21/3/2003'
# a generator will be returned by the datefinder module. I'm typecasting it to a list. Please read the note of caution provided at the bottom.
matches = list(datefinder.find_dates(input_string))

print(max(matches)+datetime.timedelta(days=1096)) 

# if len(matches) > 0:
#     # date returned will be a datetime.datetime object. here we are only using the first match.
#     date = matches[0]
#     print(date)
# else:
#     print('No dates found')

2024-07-20 00:00:00


In [11]:
from datetime import datetime
datetime.now().strftime('%m/%d/%Y')

'07/29/2021'